# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [6]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [7]:
from sklearn.model_selection import train_test_split

X = adult_dt.drop('income', axis=1)
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler


num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

num_transforms = Pipeline(steps=[
    ('KNNimputer', KNNImputer(n_neighbors = 7,weights='distance')),
    ('scaler', RobustScaler())
])

cat_transforms= Pipeline(steps=[
    ('simpleimputer',SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist',drop='if_binary'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, num_cols), 
        ('cat_transforms', cat_transforms, cat_cols)   
    ]
)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [9]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier',RandomForestClassifier())])

pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('KNNimputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [11]:
from sklearn.model_selection import cross_validate

pipeline.fit(X_train, Y_train)

scorers = ['neg_log_loss', 'roc_auc', 'accuracy','balanced_accuracy']

res_dict = cross_validate(pipeline, X_train, Y_train, cv=5, scoring=scorers, return_train_score=True)

res = pd.DataFrame(res_dict).assign(experiment=1)

res

c:\Users\Ryan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Ryan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
0   9.285996    0.160025          -0.356698           -0.081848      0.904884   
1   8.865003    0.172997          -0.356986           -0.081602      0.904288   
2   9.975026    0.181000          -0.345657           -0.081035      0.903675   
3  10.299030    0.171000          -0.370312           -0.082179      0.907552   
4   9.720645    0.191000          -0.393993           -0.081540      0.901918   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
0            1.0       0.850844        1.000000                0.777395   
1            1.0       0.846238        0.999945                0.764521   
2            1.0       0.853883        1.000000                0.774026   
3            1.0       0.861123        1.000000                0.786804   
4            1.0       0.855638        0.999945                0.777646   

   train_balanced_accuracy  experiment  
0                 1.000000           1  
1                 0.999887           1  
2                 1.000000           1  
3                 1.000000           1  
4                 0.999887           1

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [12]:
res.sort_values(by='test_neg_log_loss')

fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
4   9.720645    0.191000          -0.393993           -0.081540      0.901918   
3  10.299030    0.171000          -0.370312           -0.082179      0.907552   
1   8.865003    0.172997          -0.356986           -0.081602      0.904288   
0   9.285996    0.160025          -0.356698           -0.081848      0.904884   
2   9.975026    0.181000          -0.345657           -0.081035      0.903675   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
4            1.0       0.855638        0.999945                0.777646   
3            1.0       0.861123        1.000000                0.786804   
1            1.0       0.846238        0.999945                0.764521   
0            1.0       0.850844        1.000000                0.777395   
2            1.0       0.853883        1.000000                0.774026   

   train_balanced_accuracy  experiment  
4                 0.999887           1  
3                 1.000000           1  
1                 0.999887           1  
0                 1.000000           1  
2                 1.000000           1

Calculate the mean of each metric. 

In [13]:

res.mean()

fit_time                   9.629140
score_time                 0.175205
test_neg_log_loss         -0.364729
train_neg_log_loss        -0.081641
test_roc_auc               0.904464
train_roc_auc              1.000000
test_accuracy              0.853546
train_accuracy             0.999978
test_balanced_accuracy     0.776078
train_balanced_accuracy    0.999955
experiment                 1.000000
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [14]:
import sklearn.metrics
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

pipeline.fit(X_train, Y_train)

y_predictor = pipeline.predict(X_test)
y_prediction = pipeline.predict_proba(X_test)

metrics = {
    'log_loss': log_loss(Y_test, y_prediction),
    'roc_auc': roc_auc_score(Y_test, y_prediction[:, 1]),
    'accuracy': accuracy_score(Y_test, y_predictor),
    'balanced_accuracy': balanced_accuracy_score(Y_test, pipeline.predict(X_test))
}

metrics

{'neg_log_loss': 0.3704988503311599,
 'roc_auc': 0.9009684728944105,
 'accuracy': 0.8526973078104207,
 'balanced_accuracy': 0.7717654240047023}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.